In [4]:
from collections import defaultdict

In [ ]:
#TODO
#remove header in csv

#20170629
#For any user A, it's hard to find a similar user.(the smallest cosine similarity is always larger than 0.5 based on LSHForest) 
#NMF trainning time is under expectation, todo

#20170706
#item-based user-based has been tried, bad result
#NMF, xgboost to do next

In [3]:
prior_order = []
train_order = []
test_order = []

with open('../csv/orders.csv') as f:
    for line in f:
        items = line.strip().split(",")
        order_id, user_id, eval_set = items[0: 3]
        if eval_set == 'prior': prior_order.append((order_id, user_id))
        elif eval_set == 'train': train_order.append((order_id, user_id))
        else: test_order.append((order_id, user_id))

In [6]:
prior_order_product = defaultdict(list)
train_order_product = defaultdict(list)

def get_order_product(res, fin):
    with open(fin) as f:
        for line in f:
            items = line.strip().split(',')
            order_id, product_id = items[0: 2]
            res[order_id].append(product_id)
get_order_product(prior_order_product, '../csv/order_products__prior.csv')
get_order_product(train_order_product, '../csv/order_products__train.csv')

In [7]:
history = defaultdict(int)

for order_id,user_id in prior_order:
    if order_id in prior_order_product:
        for product_id in prior_order_product[order_id]:
            history[(user_id, product_id)] += 1

In [34]:
product2user = defaultdict(set)

for u, p in history: product2user[p].add(u)

In [10]:
n, m  = 206210, 49689

In [15]:
from scipy.sparse import csr_matrix

row = []
col = []
val = []
for (user_id, product_id), v in history.iteritems():
    row.append(user_id)
    col.append(product_id)
    val.append(v)

X = csr_matrix((val, (row, col)), shape=(n, m))

In [170]:
def f1(prediction, expect, verbose=False):
    prediction = [int(p) for p in prediction]
    expect = set([int(e) for e in expect])
    
    if verbose:
        print(prediction)
        print(expect)
    tp = 0
    for c in prediction:
        if c in expect:
            tp += 1
    if tp != 0:
        precision = tp * 1.0 / len(predict)
        recall = tp * 1.0 / len(expect)
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = 0
    return f1

In [40]:
from sklearn.neighbors import KDTree, NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.neighbors import LSHForest

In [42]:
#user-based CF
lshf = LSHForest(random_state=2333)
lshf.fit(X)
distances, indices = lshf.kneighbors(X[0:100, :], 10)

LSHForest(min_hash_match=4, n_candidates=50, n_estimators=10, n_neighbors=5,
     radius=1.0, radius_cutoff_ratio=0.9, random_state=2333)

In [172]:
##user-based CF, search
for order_id, user_id in train_order[1: 10]:
    order_id = int(order_id)
    user_id = int(user_id)
    expect = train_order_product[str(order_id)]
    candidates = defaultdict(float)
    for dis,su in zip(distances[user_id], indices[user_id]):
        row, product_ids = X[su].nonzero()
        product_numbers = X[su][(row, product_ids)].tolist()[0]
        for product_id, product_number in zip(product_ids, product_numbers):
            gap = product_number - X[int(user_id), int(product_id)]
            if gap > 0: gap = 1
            elif gap < 0: gap = -1
            else: gap = 1
            candidates[product_id] += gap * dis
    candidates = [(k, v) for k, v in candidates.iteritems()]
    candidates.sort(key=lambda o: o[1], reverse=True)
    print(f1([c[0] for c in candidates[0: 10]], expect))

0.0487804878049
0
0
0.0714285714286
0.0625
0
0
0
0.125


In [180]:
#based on self
s, c = 0, 0
for order_id, user_id in train_order[0: 100]:
    row, product_ids = X[user_id].nonzero()
    product_numbers = X[su][(row, product_ids)].tolist()[0]
    expect = train_order_product[str(order_id)]
    ps = [(pid, pnumber) for pid, pnumber in zip(product_ids, product_numbers)]
    ps.sort(key=lambda o: o[1], reverse=True)
    s += f1([p[0] for p in ps[0: 10]], expect)
    c += 1
print(s / c)

0.13194618235


In [191]:
#item based
T = X.T
lshf = LSHForest(random_state=2333)
lshf.fit(T)

1.56726408005


In [193]:
#item based search
start_time = time.time()
distances, indices = lshf.kneighbors(T[0: 1000, :], 3)
print(time.time() - start_time)

13.3247950077


In [190]:
##item-based calculate
for order_id, user_id in train_order[1: 2]:
    order_id = int(order_id)
    user_id = int(user_id)
    expect = train_order_product[str(order_id)]
    
    candidates = defaultdict(float)
    row, product_ids = X[user_id].nonzero()
    for product_id in product_ids:
            gap = product_number - X[int(user_id), int(product_id)]
            if gap > 0: gap = 1
            elif gap < 0: gap = -1
            else: gap = 1
            candidates[product_id] += gap * dis
    candidates = [(k, v) for k, v in candidates.iteritems()]
    candidates.sort(key=lambda o: o[1], reverse=True)
    print(f1([c[0] for c in candidates[0: 10]], expect))

array([ 0.        ,  0.72774849,  0.8619122 ,  0.87007009,  0.88881868,
        0.97467249,  0.98913076,  0.99145754,  0.99470299,  0.99558198])

In [67]:
from sklearn.decomposition import NMF

In [68]:
nmf = NMF(n_components=10, init='random', random_state=0)

In [75]:
nmf.fit(X)

NMF(alpha=0.0, beta=1, eta=0.1, init='random', l1_ratio=0.0, max_iter=200,
  n_components=10, nls_max_iter=2000, random_state=0, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [156]:
from sklearn.metrics import f1_score

f1_score([1, 1, 1, 0, 0], [0, 0, 0, 1, 0], average='binary')

0.0